# 神经网络架构

## AlexNet  (63.3% - 2012) 
成就：
  第一个在ImageNet上跑起来的神经网络，在当年的竞赛中成绩大幅度领先第二名。
创新：
1. 2张GTX580 3G显存上训练百万级别的数据，在模型训练上做了一些工程的改进，现在单张A100显存能到80G，足以见当年的艰难。
2. 使用大卷积（11x11、5x5）和 全连接层，事实证明潮流是一个cycle，现在大卷积又开始流行起来了= =。
3. RELU：非线性激活单元，直到现在依然很流。
4. Dropout：防止过拟合，有模型ensemble的效果，后续应用广泛。
5. Local Response Normalization：一种正则化方法帮助模型更好的训练，后续基本没人用，大家可以阅读原文了解下。

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

##  VGG  (74.5% - 2014) 
成就： 
  ImageNet成绩大幅超过AlexNet，引领了未来网络朝着深度加深的方向进行。
创新： 
1. 使用3X3卷积核代替11X11, 5X5，将网络的深度做进一步加深的同时引入更多的非线性层。

In [ ]:
import torch.nn as nn


cfg = {
    "vgg11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "vgg19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
}


class VGG(nn.Module):
    def __init__(self, vgg_name, num_outputs=10):
        super().__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, num_outputs)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == "M":
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [
                    nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                    nn.BatchNorm2d(x),
                    nn.ReLU(inplace=True),
                ]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

##  ResNet (78.6% - 2015)
成就： 
  利用残差结构使得网络达到了前所未有的深度同时性能继续提升、同时使损失函数平面更加光滑
创新： 
1. 残差网络

In [ ]:
class ResNetBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, out_planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        # print(f"in_planes : {in_planes} | self.expansion * out_planes : {self.expansion * out_planes}")
        if stride != 1 or in_planes != self.expansion * out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        # print(f"  conv 1: {out.shape}")
        out = self.bn2(self.conv2(out))
        # print(f"  conv 2: {out.shape}")
        out += self.shortcut(x)  #! key  关键代码就这一行！！！！！！！
        # print(f"shortcut: {out.shape}")
        out = F.relu(out)  # 然后一起relu
        # print("===" * 10)
        return out

##  Inception (80.0% - 2016)  GoogLeNet
创新：
1. 使用多尺度卷积核来提取信息，V1-V4基本就是在做这件事，无非是不断的优化性能。
2. 提出了Label Smoothing，这个东西比赛用的挺多的。

## DenseNet (77.8% - 2016) 
创新：
1. 利用DenseBlock进行新特征的探索和原始特征的多次重用

In [ ]:
class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4 * growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out, x], 1) #! key
        return out

## ResNext (80.9% - 2016) 
创新：
1. 提出Group的概念、利用Group增加特征的丰富度和多样性，类似multi-head attention。

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    """Grouped convolution block."""

    expansion = 2

    def __init__(self, in_planes, cardinality=32, bottleneck_width=4, stride=1):
        super(Block, self).__init__()
        group_width = cardinality * bottleneck_width
        self.conv1 = nn.Conv2d(in_planes, group_width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(group_width)
        self.conv2 = nn.Conv2d(
            group_width, group_width, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False
        )  #! key
        self.bn2 = nn.BatchNorm2d(group_width)
        self.conv3 = nn.Conv2d(group_width, self.expansion * group_width, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * group_width)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * group_width:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * group_width, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * group_width),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

## Res2Net (81.2% - 2016) 
亮点：
1. 将多特征图的处理从layer并行的形势改为hierarchical

In [ ]:
# 因为修改了特征图的交互为hierarchical，所以代码有点多
class Bottle2neck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, baseWidth=26, scale=4, stype="normal"):
        """Constructor
        Args:
            inplanes: input channel dimensionality
            planes: output channel dimensionality
            stride: conv stride. Replaces pooling layer.
            downsample: None when stride = 1
            baseWidth: basic width of conv3x3
            scale: number of scale.
            type: 'normal': normal set. '': first block of a new .
        """
        super(Bottle2neck, self).__init__()
        # todo baseWidth, width, scale的含义
        width = int(math.floor(planes * (baseWidth / 64.0)))
        print(f"width : {width}")
        self.conv1 = nn.Conv2d(inplanes, width * scale, kernel_size=1, bias=False)
        print(f"width * scale : {width * scale}")
        self.bn1 = nn.BatchNorm2d(width * scale)

        # nums的含义
        if scale == 1:
            self.nums = 1
        else:
            self.nums = scale - 1
            
        # todo stype的含义
        if stype == "":
            self.pool = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)

        # 这里似乎是核心改进点
        convs = []
        bns = []
        for i in range(self.nums):
            convs.append(nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, bias=False))
            bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)
        print(f"convs : {len(convs)}")
        self.conv3 = nn.Conv2d(width * scale, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stype = stype
        self.scale = scale
        self.width = width
        print("============= init finish =============")

    def forward(self, x):
        residual = x
        print(f"x : {x.shape}")
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        print(f"conv1 : {out.shape}")        

        spx = torch.split(out, self.width, 1)
        for i in spx:
            print(i.shape)
        print(f"len(spx) : {len(spx)}")
        for i in range(self.nums):
            if i == 0 or self.stype == "":
                sp = spx[i]
            else:
                sp = sp + spx[i]
            
            print(f"sp : {sp.shape}")
            sp = self.convs[i](sp)
            sp = self.relu(self.bns[i](sp))
            if i == 0:
                out = sp
            else:
                out = torch.cat((out, sp), 1) # 相当于y2-y3-y4
        if self.scale != 1 and self.stype == "normal":
            out = torch.cat((out, spx[self.nums]), 1) # 相当于y1的部分
        elif self.scale != 1 and self.stype == "":
            out = torch.cat((out, self.pool(spx[self.nums])), 1)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

##  SENet (81.3% - 2017) 
创新：
1. 提出SELayer、利用可插拔的SELayer调节不同Channel的重要性，和Attention效果类似。

In [ ]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)  # 将(b,c,1,1)转换为(b,c)
        y = self.fc(y).view(b, c, 1, 1)  # 将(b,c)转换为(b,c,1,1), 方便做attention
        return x * y.expand_as(x)


## DPN (80.1% - 2017) 
创新：
1. 将resnet和densenet的思想做了结合。

In [ ]:
class DPNBottleneck(nn.Module):
    def __init__(self, last_planes, in_planes, out_planes, dense_depth, stride, first_layer):
        super(DPNBottleneck, self).__init__()
        self.out_planes = out_planes
        self.dense_depth = dense_depth

        self.conv1 = nn.Conv2d(last_planes, in_planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=32, bias=False)
        self.bn2 = nn.BatchNorm2d(in_planes)
        self.conv3 = nn.Conv2d(in_planes, out_planes + dense_depth, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes + dense_depth)

        self.shortcut = nn.Sequential()
        if first_layer:
            self.shortcut = nn.Sequential(
                nn.Conv2d(last_planes, out_planes + dense_depth, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes + dense_depth),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        x = self.shortcut(x)
        d = self.out_planes
        out = torch.cat(
            [x[:, :d, :, :] + out[:, :d, :, :], x[:, d:, :, :], out[:, d:, :, :]], 1
        )  #! key  关键代码！！！   + is residual, cat is dense
        out = F.relu(out)
        return out

##  DLA (78.0% - 2019)
创新： 
1. 采用IDA和HDA两种结构来进一步提炼conv的表达

In [ ]:
"""DLA in PyTorch.
Reference:
    Deep Layer Aggregation. https://arxiv.org/abs/1707.06484
"""
import torch
import torch.nn as nn
import torch.nn.functional as F

# dla相当于只有HDA + IDA


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Root(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1):
        super(Root, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size - 1) // 2, bias=False
        )
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, xs):
        x = torch.cat(xs, 1)
        out = F.relu(self.bn(self.conv(x)))
        return out


class Tree(nn.Module):
    def __init__(self, block, in_channels, out_channels, level=1, stride=1):
        super(Tree, self).__init__()
        self.level = level
        if level == 1:
            self.root = Root(2 * out_channels, out_channels)
            self.left_node = block(in_channels, out_channels, stride=stride)
            self.right_node = block(out_channels, out_channels, stride=1)
        else:
            self.root = Root((level + 2) * out_channels, out_channels)
            for i in reversed(range(1, level)):
                subtree = Tree(block, in_channels, out_channels, level=i, stride=stride)
                self.__setattr__("level_%d" % i, subtree)
            self.prev_root = block(in_channels, out_channels, stride=stride)
            self.left_node = block(out_channels, out_channels, stride=1)
            self.right_node = block(out_channels, out_channels, stride=1)

    def forward(self, x):
        xs = [self.prev_root(x)] if self.level > 1 else []
        for i in reversed(range(1, self.level)):
            level_i = self.__getattr__("level_%d" % i)
            x = level_i(x)
            xs.append(x)
        x = self.left_node(x)
        xs.append(x)
        x = self.right_node(x)
        xs.append(x)
        out = self.root(xs)
        return out


class DLA(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=10):
        super(DLA, self).__init__()
        self.base = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False), nn.BatchNorm2d(16), nn.ReLU(True)
        )

        self.layer1 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1, bias=False), nn.BatchNorm2d(16), nn.ReLU(True)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False), nn.BatchNorm2d(32), nn.ReLU(True)
        )

        self.layer3 = Tree(block, 32, 64, level=1, stride=1)
        self.layer4 = Tree(block, 64, 128, level=2, stride=2)
        self.layer5 = Tree(block, 128, 256, level=2, stride=2)
        self.layer6 = Tree(block, 256, 512, level=1, stride=2)
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def test():
    net = DLA()
    print(net)
    x = torch.randn(1, 3, 32, 32)
    y = net(x)
    print(y.size())


if __name__ == "__main__":
    test()

## EfficientNet v1 v2  (86.8% - 2021) 
创新：是AutoDL在深度学习上一次非常成功的尝试
1. EfficientNet V1 uniformly scales all three dimensions(width, depth, resolution) with a fixed ratio。
2. EfficientNet V1 加入一些新block，扩大了搜索空间，并且不是equally scaling up every 。

In [ ]:
from torchvision.models import efficientnet_b0
model = efficientnet_b0()